#### **Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn import metrics                           
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

**Import "Refined_csv"**

In [38]:
df_clean = pd.read_csv("Refined_data.csv")
df_clean.head()

In [38]:
trainX = df_clean[:59400].copy()
trainY= trainX.pop('status_group')

print ("trainX shape : ", trainX.shape)
print ("trainY shape : ", trainY.shape)

trainX shape :  (59400, 32)
trainY shape :  (59400,)


/home/deepfintech/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### **Sequential Feature Selection**

In [39]:
# Random forest classifier
sfs = SFS(RandomForestClassifier(criterion='gini',   #with sqrrfs 0.6948, with out sqrrfs 68.88
                          min_samples_split=5,
                          min_samples_leaf=1,
                          max_features='auto',
                          oob_score=False,
                          random_state=1,
                           n_jobs=-1
                        
                          ) , 
           k_features=trainX.shape[1], 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=5)
sfs = sfs.fit(trainX, trainY)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:  2.8min finished

[2021-09-17 06:23:01] Features: 1/32 -- score: 0.6521380471380471[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:  3.0min finished

[2021-09-17 06:26:04] Features: 2/32 -- score: 0.7217003367003366[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  2.8min finished

[2021-09-17 06:28:51] Features: 3/32 -- score: 0.7577441077441078[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [40]:
sfdf = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
sfdf

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(13,)","[0.6468013468013468, 0.6525252525252525, 0.658...",0.652138,"(ward,)",0.004813,0.003745,0.001873
2,"(13, 25)","[0.7208754208754209, 0.7204545454545455, 0.726...",0.7217,"(ward, quantity)",0.003336,0.002596,0.001298
3,"(13, 25, 28)","[0.7626262626262627, 0.7563131313131313, 0.758...",0.757744,"(ward, quantity, waterpoint_type)",0.003844,0.002991,0.001495
4,"(13, 16, 25, 28)","[0.7700336700336701, 0.7646464646464647, 0.768...",0.76734,"(ward, scheme_name, quantity, waterpoint_type)",0.002434,0.001893,0.000947
5,"(12, 13, 16, 25, 28)","[0.7788720538720538, 0.7712962962962963, 0.778...",0.775589,"(lga, ward, scheme_name, quantity, waterpoint_...",0.004149,0.003228,0.001614
6,"(12, 13, 16, 18, 25, 28)","[0.7855218855218855, 0.7795454545454545, 0.788...",0.783805,"(lga, ward, scheme_name, construction_year, qu...",0.004669,0.003633,0.001816
7,"(12, 13, 16, 18, 19, 25, 28)","[0.7951178451178451, 0.7856902356902357, 0.796...",0.791616,"(lga, ward, scheme_name, construction_year, ex...",0.004883,0.003799,0.0019
8,"(12, 13, 16, 18, 19, 23, 25, 28)","[0.7995791245791246, 0.7915824915824916, 0.798...",0.795135,"(lga, ward, scheme_name, construction_year, ex...",0.004065,0.003163,0.001581
9,"(9, 12, 13, 16, 18, 19, 23, 25, 28)","[0.798989898989899, 0.7945286195286195, 0.8011...",0.796902,"(region, lga, ward, scheme_name, construction_...",0.00346,0.002692,0.001346
10,"(9, 12, 13, 14, 16, 18, 19, 23, 25, 28)","[0.803030303030303, 0.7964646464646464, 0.8016...",0.798906,"(region, lga, ward, population, scheme_name, c...",0.003668,0.002854,0.001427


In [41]:
sfdf.avg_score.max()

0.8144276094276094

In [50]:
sfdf.loc[sfdf.avg_score==sfdf.avg_score.max()]

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
28,"(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[0.8214646464646465, 0.8122053872053872, 0.814...",0.814428,"(amount_tsh, gps_height, longitude, latitude, ...",0.004885,0.003801,0.0019


**Get columns with max score**

In [51]:
selected_features=list(sfdf.loc[28, 'feature_idx'])
selected_cols = df_clean.iloc[:,selected_features] 
print (len(selected_cols.columns), " columns")
selected_cols.columns

28  columns


Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'wpt_name',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'scheme_name', 'permit',
       'construction_year', 'extraction_type', 'extraction_type_class',
       'management_group', 'payment', 'water_quality', 'quantity', 'source',
       'source_class', 'waterpoint_type', 'decade', 'installer_category'],
      dtype='object')